In [1]:
from pprint import pprint
from threading import Lock
from langchain.chains import (
    LLMChain,
    StuffDocumentsChain,
    ReduceDocumentsChain,
    MapReduceDocumentsChain
)
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_ollama import ChatOllama
from langchain_community.document_loaders import TextLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from config import Config
from database import PromptDB
import history
from llms import get_llama

In [2]:
def mapreduce(docs, host, results, key):
    llm: ChatOllama = get_llama(host=host)

    # Map
    map_template = """
Responda com um resumo fiel de até três parágrafos dos documentos no contexto.
Responda apenas com o resumo, e mais nada.

### CONTEXTO
```
{docs}.
```
### FIM DO CONTEXTO

### RESUMO FIEL
    """
    map_prompt = ChatPromptTemplate([("human", map_template)])
    map_chain = LLMChain(llm=llm, prompt=map_prompt)


    # Reduce
    reduce_template = """
Você receberá uma sequência de resumos, combine-os mantendo a ordem e a coerência, produzindo um novo resumo.
Responda apenas com o novo resumo, e mais nada.

### CONTEXTO
```
{docs}
```
### FIM DO CONTEXTO

### RESUMO FIEL
    """
    reduce_prompt = ChatPromptTemplate([("human", reduce_template)])
    reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)


    # Takes a list of documents, combines them into a single string, and passes this to an LLMChain
    combine_documents_chain = StuffDocumentsChain(
        llm_chain=reduce_chain, document_variable_name="docs"
    )

    collapse_prompt = PromptTemplate.from_template(
        """
        Colapse o seguinte texto: {context}
        """
    )
    collapse_documents_chain = StuffDocumentsChain(
        llm_chain=LLMChain(llm=llm, prompt=collapse_prompt)
    )

    # Combines and iteratively reduces the mapped documents
    reduce_documents_chain = ReduceDocumentsChain(
        # This is final chain that is called.
        combine_documents_chain=combine_documents_chain,
        # If documents exceed context for `StuffDocumentsChain`
        collapse_documents_chain=collapse_documents_chain,
    )

    # Combining documents by mapping a chain over them, then combining results
    map_reduce_chain = MapReduceDocumentsChain(
        # Map chain
        llm_chain=map_chain,
        # Reduce chain
        reduce_documents_chain=reduce_documents_chain,
        # The variable name in the llm_chain to put the documents in
        document_variable_name="docs",
        # Return the results of the map steps in the output
        return_intermediate_steps=True,
    )

    format_template = PromptTemplate.from_template(
        '''
        Separe o texto em dois a três parágrafos, mantendo a ordem, o tom e a coerência:

        ```
        {context}
        ```

        (responda apenas com o texto separado, e nada mais)
        '''
    )
    format_chain = LLMChain(llm=llm, prompt=format_template)


    print(f"calling {host}")

    result = map_reduce_chain.invoke(docs)
    pprint(result)
    result = format_chain.invoke({'context': result['output_text']})

    print('\a')
    print(f"\nHOST {host}:\n")
    pprint(result)

    # simplificar_prompt = PromptTemplate.from_template(
    #     '''
    #     Simplifique a linguagem usada no texto. Para isso, siga as seguintes diretrizes
    #     '''
    # )

    results[key]['response'] = result['text']
    results[key]['prompt'] = [
        str(map_prompt),
        str(reduce_prompt),
        str(format_template),
    ]

    # history.save('mapreduce', '', result['text'])
    return result

In [3]:
def split_documents(file_path, page_start=None, page_end=None):
    loader = PyMuPDFLoader(file_path=file_path)
    doc = loader.load()[page_start:page_end]

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=Config.SPLITTER_CHUNK_SIZE,
        chunk_overlap=Config.SPLITTER_CHUNK_OVERLAP
    )
    docs = text_splitter.split_documents(doc)
    return docs

In [4]:
file_path = "documentos/acordaos/0600012-49_REl_28052024_1.pdf"
start_page = 2
end_page = 5
doc = split_documents(file_path, start_page, end_page)

In [5]:
results = {
    "relatorio": {},
    "voto": {},
}

In [7]:
result = mapreduce(doc, 0, results, "relatorio")

/tmp/ipykernel_39753/2941991169.py:16: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  map_chain = LLMChain(llm=llm, prompt=map_prompt)
/tmp/ipykernel_39753/2941991169.py:35: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain/
  combine_documents_chain = StuffDocumentsChain(


calling 0


/home/rafael-albuquerque/LIODS/repos/langchain_first/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/rafael-albuquerque/LIODS/repos/langchain_first/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'input_documents': [Document(metadata={'source': 'documentos/acordaos/0600012-49_REl_28052024_1.pdf', 'file_path': 'documentos/acordaos/0600012-49_REl_28052024_1.pdf', 'page': 2, 'total_pages': 12, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iText® 7.1.16 ©2000-2021 iText Group NV (AGPL-version)', 'creationDate': "D:20240604121301-03'00'", 'modDate': "D:20240604121301-03'00'", 'trapped': ''}, page_content='RECORRENTE: ANTÔNIO ILOMAR VASCONCELOS CRUZ\nADVOGADOS(AS): FRANCISCO JARDEL RODRIGUES DE SOUSA - OAB CE32787-A,\nLIDENIRA CAVALCANTE MENDONÇA VIEIRA - OAB CE0016731\nRECORRIDO: MINISTÉRIO PÚBLICO ELEITORAL\n \n \nRELATÓRIO\n \nTrata-se de Recurso Eleitoral (Id 19546295) interposto por ANTÔNIO ILOMAR VASCONCELOS CRUZ em face \nde sentença proferida pelo Juízo Eleitoral da 33ª Zona – Canindé/CE, que julgou parcialmente procedente os \npedidos da Representação Eleitoral por propaganda antecipada e condenou o ora recorrente

In [9]:
results['relatorio']['response']

'O recorrente, Antônio Ilomar Vasconcelos Cruz, interpôs recurso contra sentença que julgou parcialmente procedente os pedidos da Representação Eleitoral por propaganda antecipada e condenou o ora recorrente ao pagamento de multa.\n\nA Promotoria Eleitoral alegou que o recorrente realizou propaganda eleitoral antecipada por ocasião do lançamento de sua pré-candidatura ao cargo de prefeito, com pedido explícito de votos e divulgação em redes sociais. O recorrente contestou, afirmando que as condutas ocorreram dentro dos limites previstos na lei.\n\nO recorrente argumenta que suas ações estavam dentro da legalidade, pois não ultrapassaram os limites permitidos pela legislação eleitoral.'

In [10]:
file_path = "documentos/acordaos/0600012-49_REl_28052024_1.pdf"
start_page = 5
end_page = 11
doc = split_documents(file_path, start_page, end_page)

In [11]:
result = mapreduce(doc, 0, results, "voto")

calling 0


Token indices sequence length is longer than the specified maximum sequence length for this model (1766 > 1024). Running this sequence through the model will result in indexing errors


{'input_documents': [Document(metadata={'source': 'documentos/acordaos/0600012-49_REl_28052024_1.pdf', 'file_path': 'documentos/acordaos/0600012-49_REl_28052024_1.pdf', 'page': 5, 'total_pages': 12, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iText® 7.1.16 ©2000-2021 iText Group NV (AGPL-version)', 'creationDate': "D:20240604121301-03'00'", 'modDate': "D:20240604121301-03'00'", 'trapped': ''}, page_content='ADVOGADOS(AS): FRANCISCO JARDEL RODRIGUES DE SOUSA - OAB CE32787-A,\nLIDENIRA CAVALCANTE MENDONÇA VIEIRA - OAB CE0016731\nRECORRIDO: MINISTÉRIO PÚBLICO ELEITORAL\n \n \n \nVOTO\n \nConforme relatado, trata-se de Recurso Eleitoral interposto por ANTÔNIO ILOMAR VASCONCELOS \nCRUZ em face de sentença proferida pelo Juízo Eleitoral da 33ª Zona – Canindé/CE, que julgou \nparcialmente procedente os pedidos da Representação Eleitoral por propaganda antecipada e condenou o ora \nrecorrente ao pagamento de multa no valor de R$ 15

In [12]:
results['voto']['response']

'O Tribunal Superior Eleitoral (TSE) negou provimento ao recurso eleitoral interposto por Antônio Ilomar Vasconcelos Cruz, condenando-o ao pagamento de multa em R$ 15.000,00.\n\nA decisão considerou que o evento político realizado pelo pré-candidato configurou propaganda eleitoral extemporânea irregular, pois contou com estrutura de palco, som e iluminação, e teve como objetivo apresentar as propostas do pré-candidato à população de Canindé.\n\nAlém disso, o uso do jingle de campanha "faz o i" e as mensagens publicadas nas redes sociais do pré-candidato configuraram um pedido explícito de voto, violando a paridade de armas entre os concorrentes.'

In [17]:
f"## Relatório\n\n{results['relatorio']['response']}\n\n## Voto\n\n{results['voto']['response']}"

'## Relatório\n\nO recorrente, Antônio Ilomar Vasconcelos Cruz, interpôs recurso contra sentença que julgou parcialmente procedente os pedidos da Representação Eleitoral por propaganda antecipada e condenou o ora recorrente ao pagamento de multa.\n\nA Promotoria Eleitoral alegou que o recorrente realizou propaganda eleitoral antecipada por ocasião do lançamento de sua pré-candidatura ao cargo de prefeito, com pedido explícito de votos e divulgação em redes sociais. O recorrente contestou, afirmando que as condutas ocorreram dentro dos limites previstos na lei.\n\nO recorrente argumenta que suas ações estavam dentro da legalidade, pois não ultrapassaram os limites permitidos pela legislação eleitoral.\n\n## Voto\n\nO Tribunal Superior Eleitoral (TSE) negou provimento ao recurso eleitoral interposto por Antônio Ilomar Vasconcelos Cruz, condenando-o ao pagamento de multa em R$ 15.000,00.\n\nA decisão considerou que o evento político realizado pelo pré-candidato configurou propaganda eleit